#### Выводы по анализу данных
1) датасет 0 :
    - данные распределены нормально, много выбросов в признаке f2 - значения менее -6 и более 11
2) датасет 1:
    - признаки f0, f1 распределены нормально, но у признака f1 много выборосов - значения менее -19 и более 9
    - признак f2 скорее дискретный, принимает в основном значения близкие к 0,1,2,3,4 и 5 - по нему построим круговую диаграмму
    - product тоже как будто принимает довольно определенные значения, отсмотрим его отдельно

3) датасет 2:
    - все признаки распределены нормально, большое число выбросов в f0-f2
    - f0 и f1 выбросы менее -4.2 и более 4.6
    - f2 выбросы менее -6 и более 11.5

In [2]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/autos.csv')
except FileNotFoundError:
    data = pd.read_csv('/Users/tony_builder/Documents/GitHub/studying/yandex_practicum/12_numeric_methods/data/autos.csv')

data.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [3]:
data.shape

(354369, 16)

In [4]:
data[(data.Brand == 'bmw') & (data.Model == 'corsa')]

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen


In [5]:
data[data.Power>=10000][['Price', 'Brand', 'Model', 'Kilometer', 'Power', 'RegistrationYear',
       'RegistrationMonth', 'Gearbox', 'VehicleType', 'FuelType', 'Repaired',
       'PostalCode', 'DateCrawled', 'DateCreated', 'LastSeen']]\
       .sort_values('Power', ascending=False)

,Price,Brand,Model,Kilometer,Power,RegistrationYear,RegistrationMonth,Gearbox,VehicleType,FuelType,Repaired,PostalCode,DateCrawled,DateCreated,LastSeen
219584,4300,mercedes_benz,clk,150000,20000,1999,1,auto,coupe,petrol,no,19406,2016-03-09 15:43:53,2016-03-09 00:00:00,2016-03-30 08:47:08
299180,1500,bmw,5er,150000,19312,1997,1,manual,wagon,NaN,no,21129,2016-03-09 08:53:11,2016-03-09 00:00:00,2016-04-07 11:44:34
114106,9999,bmw,1er,125000,19211,2006,0,manual,sedan,gasoline,NaN,45964,2016-04-03 07:36:21,2016-04-03 00:00:00,2016-04-07 10:16:10
132485,2100,bmw,5er,150000,19208,2001,5,manual,wagon,NaN,yes,59556,2016-03-25 14:55:49,2016-03-25 00:00:00,2016-03-30 05:16:18
63986,3250,opel,omega,150000,17932,2001,6,auto,sedan,petrol,NaN,86641,2016-03-28 11:49:56,2016-03-28 00:00:00,2016-04-06 14:17:21
196532,6000,bmw,5er,150000,17700,2004,7,auto,wagon,gasoline,no,26316,2016-03-20 16:51:48,2016-03-20 00:00:00,2016-04-06 22:45:56
150676,1700,audi,80,5000,17410,1995,0,manual,wagon,petrol,yes,23562,2016-03-09 18:57:50,2016-03-09 00:00:00,2016-03-15 07:17:06
187330,1749,bmw,3er,5000,17019,1989,12,manual,wagon,petrol,NaN,89542,2016-03-17 19:50:23,2016-03-17 00:00:00,2016-03-26 00:46:59
175422,10900,audi,a3,5000,17011,2007,7,auto,sedan,gasoline,NaN,12355,2016-03-19 11:38:23,2016-03-19 00:00:00,2016-03-21 15:20:06
216993,11950,audi,a4,20000,16312,2008,4,manual,convertible,petrol,no,51647,2016-03-10 08:57:25,2016-03-10 00:00:00,2016-03-17 09:16:19


с ручной коробкой передач

In [6]:
import pandas as pd

# Пример DataFrame с разными типами данных в столбце
data = {'Столбец': ['строка', ['список'], 'еще строка', {'словарь': 'значение'}]}
df = pd.DataFrame(data)

# Проверяем тип данных в столбце 'Столбец'
result = df['Столбец'].apply(lambda x: type(x) in  [list])

print(result)


0    False
1     True
2    False
3    False
Name: Столбец, dtype: bool


In [7]:
'<25'

'<25'